In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


In [ ]:
# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

DEBUG:lib.lab_lib:hello


: 

: 

: 

: 

: 

# 2022年5月28日

LULESHのプロファイルから予測をする

In [ ]:
date :str = "2022年6月1日"

input_rawDF :pd.DataFrame = return_rawDFinLULESH(
    processes=lulesh_processes,
    iterations=lulesh_iterations,
    sizes=lulesh_sizes,
    csvDirPath=csvDirPath,
)

input_rawDF = input_rawDF.rename(columns={"Name":"functionName"})

input_list_expVarNames :list[str] = ["process", "iteration", "size"]
input_list_resVarNames :list[str] = ["#Call"]

model_names: list[str] = [
    "modelLin",
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations"
    # "modelBasicTree",
]

# 集合演算を用いて説明変数・目的変数が入力DFに存在していることを確認
if input_list_expVarNames == [] or (set(input_list_expVarNames).issubset(set(input_rawDF.columns.tolist())) == False):
    warnings.warn(f"説明変数の指定に誤りがあります")
    # return False
if input_list_resVarNames == [] or (set(input_rawDF.columns.tolist()) == False):
    warnings.warn(f"問題変数の指定に誤りがあります")
    # return False

function_names :list[str] = list(set(input_rawDF["functionName"].tolist()))
if len(function_names) == 0:
    warnings.warn("与えられたDFに関数名がありません")
    # return False

result_series_list :list[pd.Series] = []

for function_name in function_names:
    # 関数ごとの生データ
    input_rawDF_per_function = input_rawDF[input_rawDF["functionName"] == function_name]
    # モデルの構築
    models = Models(
        inputDF = input_rawDF_per_function,
        expVarColNames=input_list_expVarNames,
        resVarColNames=input_list_resVarNames,
        targetDF=None,
        modelNames=model_names,
    )

    models.setUpDataBeforeCalcLr()
    models.calcLr()
    models.calcMAPE()

    dictCalcedMAPE = models.returnCalculatedMAPE()

    # 算出されたMAPEの数値をfloatにする
    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = float(dictCalcedMAPE[key])

    # 関数ごとの結果に格納
    dict_for_series :dict = copy.deepcopy(dictCalcedMAPE)
    dict_for_series["functionName"] = function_name

    series = pd.Series(dict_for_series)
    result_series_list.append(series)

resultDF :pd.DataFrame = pd.DataFrame(result_series_list)
resultDF = addLowestMAPEColumn(inputDF=resultDF, model_name_list=model_names, version=2)


/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estima

: 

: 

: 

: 

: 

In [ ]:
resultDF

,modelLin,modelIp,modelLog,modelProcessDividedByProblemSize,modelProblemSizeDividedByProcess,modelLinAndIp,modelLinAndLog,modelIpAndLin,modelIpAndLog,modelLogAndLin,modelLogAndIp,modelLinearSumOf2elementCombination,modelLinearSumOfElementCombinations,functionName,最低値
0,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,void_Domain::SetupBoundaryConditions(Int_t),0.000000e+00
1,1.646390e-13,177.412047,115.178571,7.668323e+01,7.655747e+01,177.412047,115.178571,9.075633e-15,115.178571,4.070818e-14,177.412047,117.100765,2.199475e-12,void_CommMonoQ(Domain,9.075633e-15
2,5.597246e+01,193.662948,151.023292,8.183808e+01,8.405442e+01,193.166473,142.326222,6.802352e+01,148.908970,7.353575e+01,192.710753,104.783416,3.866607e+00,MPI_Irecv(),3.866607e+00
3,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,void_Domain::SetupElementConnectivities(Int_t),0.000000e+00
4,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,void_Domain::SetupCommBuffers(Int_t),0.000000e+00
5,1.799660e-13,192.995792,127.512160,7.808887e+01,7.752953e+01,192.995792,127.512160,9.971133e-15,127.512160,4.489668e-14,192.995792,128.765533,2.410129e-12,MPI_Allreduce(),9.971133e-15
6,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,MPI_Barrier(),0.000000e+00
7,9.897074e+03,11008.061076,12824.894431,1.406233e+02,1.407564e+02,11008.061076,12824.894431,9.897074e+03,12824.894431,9.897074e+03,11008.061076,4521.029402,1.327401e+03,Real_t_CalcElemVolume(const,1.406233e+02
8,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,MPI_Init(),0.000000e+00
9,0.000000e+00,0.000000,0.000000,2.308030e-08,1.862710e-08,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,void_Domain::CreateRegionIndexSets(Int_t_Int_t),0.000000e+00


: 

: 

: 

: 

: 

In [ ]:
resultDF.mean()

/tmp/ipykernel_345/2794288614.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  resultDF.mean()


modelLin                               324.173667
modelIp                                411.497704
modelLog                               454.526334
modelProcessDividedByProblemSize        40.081234
modelProblemSizeDividedByProcess        35.645728
modelLinAndIp                          421.093841
modelLinAndLog                         455.735629
modelIpAndLin                          315.660772
modelIpAndLog                          446.710080
modelLogAndLin                         323.795580
modelLogAndIp                          419.026472
modelLinearSumOf2elementCombination    194.468150
modelLinearSumOfElementCombinations     41.843775
最低値                                      4.756974
dtype: float64

: 

: 

: 

: 

: 

In [ ]:
DF_funcName_lowestMAPE :pd.DataFrame = resultDF.filter(items=["functionName", "最低値"])
DF_funcName_lowestMAPE = DF_funcName_lowestMAPE.rename(columns={"functionName":"関数名", "最低値":"最低MAPE"})

print(DF_funcName_lowestMAPE.style.format(precision=2).to_latex(
    caption=f"ベンチマークプログラムLULESHの関数における最低MAPE",
    label=f"{date}lowestMAPEinLULESH"
))

\begin{table}
\caption{ベンチマークプログラムLULESHの関数における最低MAPE}
\label{2022年6月1日lowestMAPEinLULESH}
\begin{tabular}{llr}
 & 関数名 & 最低MAPE \\
0 & void_Domain::SetupBoundaryConditions(Int_t) & 0.00 \\
1 & void_CommMonoQ(Domain & 0.00 \\
2 & MPI_Irecv() & 3.87 \\
3 & void_Domain::SetupElementConnectivities(Int_t) & 0.00 \\
4 & void_Domain::SetupCommBuffers(Int_t) & 0.00 \\
5 & MPI_Allreduce() & 0.00 \\
6 & MPI_Barrier() & 0.00 \\
7 & Real_t_CalcElemVolume(const & 140.62 \\
8 & MPI_Init() & 0.00 \\
9 & void_Domain::CreateRegionIndexSets(Int_t_Int_t) & 0.00 \\
10 & MPI_Waitall() & 0.00 \\
11 & StrToInt & 0.00 \\
12 & void_VerifyAndWriteFinalOutput(Real_t_Domain & 0.00 \\
13 & void_CommSyncPosVel(Domain & 0.00 \\
14 & MPI_Reduce() & 0.00 \\
15 & void_Domain::BuildMesh(Int_t_Int_t_Int_t) & 0.00 \\
16 & void_Domain::~Domain() & 0.00 \\
17 & void_CalcKinematicsForElems(Domain & 0.00 \\
18 & int_main(int_char_**) & 0.00 \\
19 & MPI_Wait() & 3.87 \\
20 & void_Domain::Domain(Int_t_Index_t_Index_t_Index_t_In

: 

: 

: 

: 

: 

# 2022年5月25日～

重み付き平均の実装

todos
* ✅✅"modelLin":ModelLinForMultipleRegression
* ✅✅"modelIp":ModelIpForMultipleRegression
* ✅✅"modelLog":ModelLogForMultipleRegression
* ✅✅"modelLinAndIp":ModelMultipleEquationForMultipleRegression
* ✅✅"modelLinAndLog":ModelMultipleEquationForMultipleRegression
* ✅✅"modelIpAndLin":ModelMultipleEquationForMultipleRegression
* ✅✅"modelIpAndLog":ModelMultipleEquationForMultipleRegression
* ✅✅"modelLogAndLin":ModelMultipleEquationForMultipleRegression
* ✅✅"modelLogAndIp":ModelMultipleEquationForMultipleRegression
* ✅✅"modelProcessDividedByProblemSize":Model_ProcessesDevidedByProblemSize_ForMultipleRegression
* ✅✅"modelProblemSizeDividedByProcess":Model_ProblemSizeDevidedByProcesses_ForMultipleRegression
* ✅✅"modelLinearSumOf2elementCombination":Model_LinearSumOf2elementCombination_ForMultipleRegression
* ✅✅"modelLinearSumOfElementCombinations":Model_LinearSumOfElementCombinations_ForMultipleRegression

* `class Models`での対応
    * ✅models.calcWeightedMAPE()
    * ✅models.returnCalculatedMAPE()

In [ ]:
date :str = "2022年6月1日"

input_rawDF :pd.DataFrame = return_rawDFinLULESH(
    processes=lulesh_processes,
    iterations=lulesh_iterations,
    sizes=lulesh_sizes,
    csvDirPath=csvDirPath,
)

input_rawDF = input_rawDF.rename(columns={"Name":"functionName"})

input_list_expVarNames :list[str] = ["process", "iteration", "size"]
input_list_resVarNames :list[str] = ["#Call"]

model_names: list[str] = [
    "modelLin",
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations"
    # "modelBasicTree",
]

# 集合演算を用いて説明変数・目的変数が入力DFに存在していることを確認
if input_list_expVarNames == [] or (set(input_list_expVarNames).issubset(set(input_rawDF.columns.tolist())) == False):
    warnings.warn(f"説明変数の指定に誤りがあります")
    # return False
if input_list_resVarNames == [] or (set(input_rawDF.columns.tolist()) == False):
    warnings.warn(f"問題変数の指定に誤りがあります")
    # return False

function_names :list[str] = list(set(input_rawDF["functionName"].tolist()))
if len(function_names) == 0:
    warnings.warn("与えられたDFに関数名がありません")
    # return False

result_series_list :list[pd.Series] = []

for function_name in function_names:
    # 関数ごとの生データ
    input_rawDF_per_function = input_rawDF[input_rawDF["functionName"] == function_name]
    # モデルの構築
    models = Models(
        inputDF = input_rawDF_per_function,
        expVarColNames=input_list_expVarNames,
        resVarColNames=input_list_resVarNames,
        targetDF=None,
        modelNames=model_names,
    )

    models.setUpDataBeforeCalcLr()
    models.calcLr()
    models.calcWeightedMAPE()

    dictCalcedWeightedMAPE = models.returnCalculatedWeightedMAPE()

    # 算出されたMAPEの数値をfloatにする
    for key in dictCalcedMAPE.keys():
        dictCalcedWeightedMAPE[key] = float(dictCalcedWeightedMAPE[key])

    # 関数ごとの結果に格納
    dict_for_series :dict = copy.deepcopy(dictCalcedWeightedMAPE)
    dict_for_series["functionName"] = function_name

    series = pd.Series(dict_for_series)
    result_series_list.append(series)

resultDF :pd.DataFrame = pd.DataFrame(result_series_list)

: 

: 

: 

: 

# 2022年4月17日～

次のような表を作成する

採用される割合 (MAPE の最大値 [%] ，MAPE の最小値 [%]) [%]

| ベンチマークプログラム名 | 線形モデル               | 対数モデル               | 反比例モデル              |
|--------------|---------------------|---------------------|---------------------|
| str          | float(float, float) | float(float, float) | float(float, float) |


目標となるのは一気にこのベンチマークプログラムを作成することだが、既存のライブラリ関数などを利用し、まずはベンチマークごとに作成可能にする。

メモ

## 実装予定

1. 行方向に最小値を検出
2. 最小値以外をNaNに変更
3. 列方向に最小値と最大値を検出

## 
